### Notebook to parse text files to produce cleaned text of RAD decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Appeal Division Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rad/>.

Notes:

(1) Data Source: Immigration and Refugee Board. In the Fall of 2022, the IRB added the Refugee Law Laboratory to their email distribution list for legal publishers of RAD decisions. The RLL therefore receives new RAD cases as they are released for publication by the IRB. Also, in the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). 

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores for coding the parsing scripts.


# Installing Libraries

In [ ]:
#!pip install langdetect
#!pip install regex
#!pip install dask

# Importing Libraries

In [1]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect, DetectorFactory
from difflib import get_close_matches
import json
from tqdm import tqdm
import chardet
import dask.bag as db
from dask.diagnostics import ProgressBar
import pathlib
from pathlib import Path
from datetime import datetime, timezone


## Declaring Constant
Here, we specify the directories containing our data files.

In [2]:
#DATA_DIRS = ["../RAD Decisions TEXT", "../IRB Decisions - Initial Request - TEXT"]

# For SR:
DATA_DIRS = ["d:/RAD Decisions TEXT/", "d:/IRB Decisions - Initial Request - TEXT/"]

# TEMMP for testing:

#DATA_DIRS = ["d:/RAD Decisions TEXT/"]


In [3]:
# set seed for langdetect for consistent results and reproducibility
DetectorFactory.seed = 42



## Language Detection
This function determines the language of a given text.

In [4]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None


## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [5]:
def original_match_date_patterns(content):
    patterns = {
        "custom": (r"Date (?:of decision|de la décision)\s*\n\s*([A-Za-z]+)\s+(\d{1,2})\.\s*(\d{4})", lambda m: [m.group(1), m.group(2), m.group(3)]),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split()),
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)
    return None

def match_date_patterns(content):
    # 1. Try your proven patterns first
    result = original_match_date_patterns(content)
    if result:
        return result

    # 2. Fallback: Check lines after the label
    lines = content.splitlines()
    label_regex = re.compile(
        r'^\s*Date (of decision|de la décision)(\s*and reasons|\s*et des motifs)?\s*$', re.IGNORECASE)

    date_regexes = [
        # French style: 1er mai 2015, 24 juillet 2018, Le 24 juillet 2018
        re.compile(r'^(Le\s+)?(\d{1,2}|1er)\s+\w+\s+\d{4}$', re.IGNORECASE),
        # English style: January 21, 2019 or November 5th, 2014
        re.compile(r'^[A-Za-z]+\s+\d{1,2}(st|nd|rd|th)?\,?\s+\d{4}$'),
        # Dashes (rare but sometimes): 10-12-2018
        re.compile(r'^\d{1,2}-\d{1,2}-\d{4}$'),
    ]

    for idx, line in enumerate(lines):
        if label_regex.match(line):
            # Look ahead at the next 3 non-empty lines
            lookahead = 0
            for j in range(idx+1, min(idx+5, len(lines))):
                candidate = lines[j].strip()
                if not candidate:
                    continue
                lookahead += 1
                for reg in date_regexes:
                    m = reg.match(candidate)
                    if m:
                        cleaned = candidate.replace("Le ", "").replace(",", "")
                        return cleaned.split()
                if lookahead >= 3:
                    break
    
    return None

## Date Formatter
Takes detected regular expression and converts into one common format

In [6]:
french_to_english = {
        'janvier': 'January', 'fevrier': 'February', 'mars': 'March', 'avril': 'April',
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'aout': 'August',
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'decembre': 'December'
}

def correct_month_name(misspelled_month, possibilities=['Janvier','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    if correct_months:
        corrected_month = correct_months[0]
        # Check if the corrected month is in the French to English mapping
        return french_to_english.get(corrected_month.lower(), corrected_month)
    else:
        return misspelled_month

def correct_year_typo(year):
    if len(year) == 3 and year.startswith("0"):
        return "20" + year[1:]
    return year

def correct_year_typo(year):
    """Corrects year format typos."""
    return "20" + year[1:] if len(year) == 3 and year.startswith("0") else year

def process_numeric_format(parts):
    """Processes numeric date format 'dd-mm-yyyy'."""
    day, month, year = parts[0].split('-')
    year = correct_year_typo(year)
    return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

def process_day_first_format(parts, french_month_mapping):
    """Processes dates in 'day month year' format, French or English."""
    day = 1 if parts[0].lower() == '1er' else int(parts[0])

    month = ''
    # Check if month and year are concatenated
    if len(parts) == 2 and not parts[1].isdigit():
        month_year_str = parts[1]
        for i in range(1, len(month_year_str)):
            if month_year_str[i:].isdigit():
                month_str, year_str = month_year_str[:i], month_year_str[i:]
                year = correct_year_typo(year_str)
                month = french_month_mapping.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str.capitalize())
                break
    else:
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        year = correct_year_typo(parts[2])

    if month in french_month_mapping:
        return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
    else:
        if isinstance(month, int):
            return datetime(int(year), month, day).date().strftime('%Y-%m-%d')
        
        corrected_month = correct_month_name(month.capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

def process_month_first_format(parts):
    """Processes month first format with possible ordinal suffix."""
    day = 0
    month = ''
    year = ''
    
    if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
        
        if parts[1].isdigit() and len(parts[1]) > 4: 
            month = parts[0]
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = year_str
            day = int(day_str)
            
        elif parts[1].isdigit()and len(parts[1]) > 3: #Year is the second entry
            month_day_str = parts[0]
            for i in range(1, len(month_day_str)):
                if not month_day_str[i].isdigit():
                    day_str, month_str = month_day_str[:i], month_day_str[i:]
                    day = int(day_str)
                    month = french_to_english.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str)
                    parts[0] = month
                    break
            year = parts[1]
        else:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)

    else:
        day = re.sub(r"[^\d]", "", parts[1])
        day = int(day) if day.isdigit() else 1
        year = correct_year_typo(parts[2])
        
    
    try:
        corrected_month = correct_month_name(parts[0].capitalize())
        return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [7]:
def process_date_parts(parts, french_month_mapping):
    """Determines the correct date processing method based on the format of the parts."""
    if '-' in parts[0]:
        return process_numeric_format(parts)
    elif parts[0].isdigit() or parts[0].lower() == '1er':
        return process_day_first_format(parts, french_month_mapping)
    else:
        return process_month_first_format(parts)

def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [8]:
def extract_rad_number(content):
    """Extracts the RAD number from the content, ignoring IAD files."""
     
    # Check for lines indicating the file should be ignored
    ignore_lines = ["IAD File",
                    "IMMIGRATION APPEAL DIVISION", 
                    "ID File", 
                    "IMMIGRATION DIVISION", 
                    "RPD File", 
                    "RPD file",
                    "REFUGEE PROTECTION DIVISION", 
                    "REFUGEE DIVISION"
                    ]
    
    
    for line in content.splitlines():
     
        if any(ignore_line in line for ignore_line in ignore_lines):       
            return None
                
        sanitized_line = ''.join(c for c in line if c.isprintable()).strip()
    
        if "RAD File" in sanitized_line :
            
            rad_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rad_number_match:
                return rad_number_match.group(1)
            
            # If RAD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rad_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rad_number_match:
                    return rad_number_match.group(1)
        
            # fallback to catch a common error

        if "SAR File" in sanitized_line :
            rad_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rad_number_match:
                return rad_number_match.group(1)
            
            # If RAD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rad_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rad_number_match:
                    return rad_number_match.group(1)
              
    return None


def process_file(file_path):

    try:
        """Processes a single file and extracts data."""

        # Use chardet to detect the encoding of the file
        with open(file_path, 'rb') as file:
            raw_data = file.read()
        encoding = chardet.detect(raw_data)['encoding']

        # Read the file with the detected encoding
        with open(file_path, 'r', errors='replace', encoding=encoding) as file:
            content = file.read()

        # Map problematic file fragments to their corresponding RAD numbers

        problem_files = {
            "MC0-01686ta.txt": "MC0-01686",
            "MC0-04831ta.txt": "MC0-04831",
            "MC0-06114tf.txt": "MC0-06114",
            "MC0-08063ta.txt": "MC0-08063",
            "MC0-08521 f.txt": "MC0-08521",
            "MC0-10174ta.txt": "MC0-10174",
            "MC0-10320tf.txt": "MC0-10320",
            "MC1-00352 tf.txt": "MC1-00352",
            "TB8-11442tf.txt": "TB8-11442",
            "TB9-10382tf.txt": "TB9-10382",
            "TB9-29568tf.txt": "TB9-29568",
            "TC0-05851tf.txt": "TC0-05851",
            "TC0-10233 a.txt": "TC0-10233",
            "TC1-04783tf.txt": "TC1-04783",
            "TC1-12335 a.txt": "TC1-12335",
            "TC1-12335 tf.txt": "TC1-12335",
            "1691090.txt": "MB4-00757",
            "1747179.txt": "MB4-02189",
            "1747181.txt": "MB4-03314",
            "1797799.txt": "MB3-03350",
            "1802396.txt": "VB4-01077",
            "1802506.txt": "VB3-02589",
            "1820441.txt": "MB4-04618",
            "1825243.txt": "MB4-02540",
            "1825447.txt": "VB4-01870",
            "2294273.txt": "MB6-05587",
            "2294277.txt": "TB5-02604",
            "2685118.txt": "VB5-00414",
            "2714254.txt": "TB5-10940",
            "2726259.txt": "TB4-03059",
            "2750270.txt": "MB4-01799",
            "2776412.txt": "TB6-08260",
            "2901493.txt": "TB8-05152",
            "2954037.txt": "TB7-20791",
            "2945923.txt": "TB7-22410",
            "2953978.txt": "TB8-17738",
            "2996850.txt": "TB7-02952",
            "3060429.txt": "MB7-00268",
            "3064579.txt": "MB8-07401",
            "3066742.txt": "TB9-05486",
            "3074204.txt": "TB8-19298",
            "3081729.txt": "TB8-04755",
            "3009899.txt": "MB7-05420",
            "3091994.txt": "TB8-22918",
            "3112189.txt": "TB9-01803",
            "3112241.txt": "MB9-01099",
            "3135686.txt": "TB8-06759",  
            "3131897.txt": "TB9-06111",
            "3169372.txt": "MB9-05816",
            "3169471.txt": "TB9-06069",
            "3169485.txt": "TB9-15836",
            "3172931.txt": "MB6-03390",
            "3173907.txt": "TB9-21518",
            "3176472.txt": "TB9-19840",
            "3178266.txt": "VB9-05507",
            "3270723.txt": "VB9-05051",
            "3386768.txt": "TB9-32990",
            "3386786.txt": "VB9-06946",
            "3399587.txt": "MB9-11935",
            "3436433.txt": "VB9-08997",
            "3444153.txt": "TC0-01391",
            "3457769.txt": "TB9-01269",
            "3514437.txt": "TB9-33272",
            "3532711.txt": "MB9-28671",
            "3546032.txt": "MB9-04262",
            "3546108.txt": "MB9-30244",
            "3546284.txt": "TB9-08176",
            "3546335.txt": "TB9-35398",
            "3567362.txt": "TB9-35626",
            "3581415.txt": "MB9-01194",
            "3594465.txt": "TB9-03384",
                        
        }

        # Check for problem files using a loop, then extract RAD number if not found
        rad_number = None
        for fragment, rad in problem_files.items():
            if fragment in file_path:
                rad_number = rad
                break

        if rad_number is None:
            rad_number = extract_rad_number(content)

        if rad_number:
            lang = detect_language(content)
            
            # address problem file, else extract document date:
            if "TC1-10832tf" in file_path:
                document_date = "2022-01-20"
            else:
                document_date = extract_document_date(content)
            
            scraped_timestamp = datetime.fromtimestamp(os.path.getmtime(file_path), tz=timezone.utc)

            return {
                'citation': rad_number,
                'citation2': '',
                'dataset': 'RAD',
                'name': '',
                'language': lang,
                'document_date': document_date,
                'url': os.path.basename(file_path),
                'scraped_timestamp':  scraped_timestamp,
                'unofficial_text': content,
            }
        return None
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        #raise Exception(f"Error processing file {file_path}: {e}") ### uncomment to see error if running in dask
        return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe.

In [9]:
# Main data processing loop (run in paraellel using Dask)

def process_file_wrapper(file_path):
    
    if not os.path.basename(file_path).startswith('~'):
        return process_file(file_path)

# Gather all file paths
file_paths = []
for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        dir_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
        file_paths.extend(dir_files)

# Create a Dask Bag from file paths
file_bag = db.from_sequence(file_paths)

# Use Dask to process files in parallel
with ProgressBar():
    results = file_bag.map(process_file_wrapper).filter(lambda x: x is not None).compute()

# Convert results to a Pandas DataFrame
df = pd.DataFrame(results)

[########################################] | 100% Completed | 246.05 s


In [10]:
df_backup = df.copy()

In [11]:
df = df_backup.copy()

### Data cleaning
Cleans data to match huggingface dataset

In [12]:
print ("Starting df", len(df))

# manually fix some document_date issues
df.loc[df['url'] == 'TB8-11442f.txt', 'document_date'] = pd.Timestamp('2020-10-15', tz='UTC')
df.loc[df['url'] == '1820460.txt', 'document_date'] = pd.Timestamp('2014-04-24', tz='UTC')
df.loc[df['url'] == '2876788.txt', 'document_date'] = pd.Timestamp('2018-04-05', tz='UTC')
df.loc[df['url'] == '3196706.txt', 'document_date'] = pd.Timestamp('2019-11-13', tz='UTC')
df.loc[df['url'] == '3196826.txt', 'document_date'] = pd.Timestamp('2019-10-28', tz='UTC')
df.loc[df['url'] == '3546110.txt', 'document_date'] = pd.Timestamp('2021-01-21', tz='UTC')
df.loc[df['url'] == '3389310.txt', 'document_date'] = pd.Timestamp('2021-04-20', tz='UTC')
df.loc[df['url'] == 'MC0-05967f.txt', 'document_date'] = pd.Timestamp('2021-04-20', tz='UTC')
df.loc[df['url'] == '3061334.txt', 'document_date'] = pd.Timestamp('2019-02-05', tz='UTC')
df.loc[df['url'] == 'MC1-12211 a.txt', 'document_date'] = pd.Timestamp('2022-08-19', tz='UTC')
df.loc[df['url'] == 'TC0-10646tf.txt', 'document_date'] = pd.Timestamp('2021-11-17', tz='UTC')
df.loc[df['url'] == 'TC1-10832 a.txt', 'document_date'] = pd.Timestamp('2022-01-20', tz='UTC')
df.loc[df['url'] == 'TC1-14698tf.txt', 'document_date'] = pd.Timestamp('2022-01-19', tz='UTC')
df.loc[df['url'] == 'TC1-14698 a.txt', 'document_date'] = pd.Timestamp('2022-01-19', tz='UTC')
df.loc[df['url'] == 'MB9-26873tf.txt', 'document_date'] = pd.Timestamp('2022-03-18', tz='UTC')
df.loc[df['url'] == 'MB9-26873a.txt', 'document_date'] = pd.Timestamp('2022-03-18', tz='UTC')
df.loc[df['url'] == 'MB9-22375tf.txt', 'document_date'] = pd.Timestamp('2020-09-03', tz='UTC')
df.loc[df['url'] == 'MB9-22375a.txt', 'document_date'] = pd.Timestamp('2020-09-03', tz='UTC')
df.loc[df['url'] == '2769752.txt', 'document_date'] = pd.Timestamp('2016-07-18', tz='UTC')
df.loc[df['url'] == '2797762.txt', 'document_date'] = pd.Timestamp('2016-07-04', tz='UTC')
df.loc[df['url'] == '2892390.txt', 'document_date'] = pd.Timestamp('2018-07-11', tz='UTC')
df.loc[df['url'] == '3567370.txt', 'document_date'] = pd.Timestamp('2021-02-01', tz='UTC')
df.loc[df['url'] == '3566569.txt', 'document_date'] = pd.Timestamp('2021-01-30', tz='UTC')
df.loc[df['url'] == '3479804.txt', 'document_date'] = pd.Timestamp('2021-01-25', tz='UTC')
df.loc[df['url'] == '3479803.txt', 'document_date'] = pd.Timestamp('2021-01-25', tz='UTC')
df.loc[df['url'] == '3479768.txt', 'document_date'] = pd.Timestamp('2021-01-11', tz='UTC')
df.loc[df['url'] == '3479767.txt', 'document_date'] = pd.Timestamp('2021-01-11', tz='UTC')
df.loc[df['url'] == '3477399.txt', 'document_date'] = pd.Timestamp('2021-01-13', tz='UTC')
df.loc[df['url'] == '3477398.txt', 'document_date'] = pd.Timestamp('2021-01-13', tz='UTC')
df.loc[df['url'] == '3444104.txt', 'document_date'] = pd.Timestamp('2021-02-02', tz='UTC')
df.loc[df['url'] == '3196782.txt', 'document_date'] = pd.Timestamp('2019-11-01', tz='UTC')
df.loc[df['url'] == '3170609.txt', 'document_date'] = pd.Timestamp('2019-05-30', tz='UTC')
df.loc[df['url'] == '3170608.txt', 'document_date'] = pd.Timestamp('2019-05-30', tz='UTC')
df.loc[df['url'] == '3170577.txt', 'document_date'] = pd.Timestamp('2019-05-10', tz='UTC')
df.loc[df['url'] == '3112268.txt', 'document_date'] = pd.Timestamp('2019-06-24', tz='UTC')
df.loc[df['url'] == '3112267.txt', 'document_date'] = pd.Timestamp('2019-06-24', tz='UTC')
df.loc[df['url'] == '3091928.txt', 'document_date'] = pd.Timestamp('2019-07-09', tz='UTC')
df.loc[df['url'] == '3061306.txt', 'document_date'] = pd.Timestamp('2018-09-27', tz='UTC')
df.loc[df['url'] == '3036997.txt', 'document_date'] = pd.Timestamp('2019-01-28', tz='UTC')
df.loc[df['url'] == '3036996.txt', 'document_date'] = pd.Timestamp('2019-01-28', tz='UTC')
df.loc[df['url'] == '3009983.txt', 'document_date'] = pd.Timestamp('2018-09-18', tz='UTC')
df.loc[df['url'] == '3009982.txt', 'document_date'] = pd.Timestamp('2018-09-18', tz='UTC')
df.loc[df['url'] == '2883691.txt', 'document_date'] = pd.Timestamp('2018-01-31', tz='UTC')
df.loc[df['url'] == '2868255.txt', 'document_date'] = pd.Timestamp('2017-12-28', tz='UTC')
df.loc[df['url'] == '2868191.txt', 'document_date'] = pd.Timestamp('2018-01-15', tz='UTC')
df.loc[df['url'] == '2868190.txt', 'document_date'] = pd.Timestamp('2018-01-15', tz='UTC')
df.loc[df['url'] == '2834981.txt', 'document_date'] = pd.Timestamp('2018-02-19', tz='UTC')
df.loc[df['url'] == '2834981.txt', 'document_date'] = pd.Timestamp('2018-02-19', tz='UTC')
df.loc[df['url'] == '1844495.txt', 'document_date'] = pd.Timestamp('2014-02-17', tz='UTC')
df.loc[df['url'] == '1844494.txt', 'document_date'] = pd.Timestamp('2014-02-17', tz='UTC')
df.loc[df['url'] == '1831965.txt', 'document_date'] = pd.Timestamp('2014-11-05', tz='UTC')
df.loc[df['url'] == '1825449.txt', 'document_date'] = pd.Timestamp('2014-12-05', tz='UTC')
df.loc[df['url'] == '1825448.txt', 'document_date'] = pd.Timestamp('2014-12-05', tz='UTC')
df.loc[df['url'] == '1797868.txt', 'document_date'] = pd.Timestamp('2015-04-13', tz='UTC')
df.loc[df['url'] == '1757086.txt', 'document_date'] = pd.Timestamp('2015-03-02', tz='UTC')
df.loc[df['url'] == '1577641.txt', 'document_date'] = pd.Timestamp('2014-06-23', tz='UTC')
df.loc[df['url'] == '1563624.txt', 'document_date'] = pd.Timestamp('2014-04-07', tz='UTC')
df.loc[df['url'] == '1563601.txt', 'document_date'] = pd.Timestamp('2014-04-07', tz='UTC')
df.loc[df['url'] == '1526721.txt', 'document_date'] = pd.Timestamp('2013-10-18', tz='UTC')
df.loc[df['url'] == '3091929.txt', 'document_date'] = pd.Timestamp('2019-06-11', tz='UTC')
df.loc[df['url'] == '2757789.txt', 'document_date'] = pd.Timestamp('2016-02-23', tz='UTC')
df.loc[df['url'] == '2901493.txt', 'document_date'] = pd.Timestamp('2018-03-16', tz='UTC')
df.loc[df['url'] == '3546053.txt', 'document_date'] = pd.Timestamp('2021-01-12', tz='UTC')
df.loc[df['url'] == '3229927.txt', 'document_date'] = pd.Timestamp('2020-01-24', tz='UTC')
df.loc[df['url'] == '3229926.txt', 'document_date'] = pd.Timestamp('2020-01-24', tz='UTC')

# remove a few rows where there is no matching other language decision, but multiple versions
df = df[~((df['url'] == '2928118.txt'))]  # fr one is missing, but en one is there + multiple
df = df[~((df['url'] == '3074046.txt'))]  # fr one is missing, but en one is there + multiple
df = df[~((df['url'] == '2746200.txt'))]  # fr one is missing, but en one is there + multiple
df = df[~((df['url'] == '3074119.txt'))]  # fr one is missing, but en one is there + multiple
df = df[~((df['url'] == '3078334.txt'))]  # fr one is missing, but en one is there + multipl
df = df[~((df['url'] == '2897171.txt'))]  # two fr same, no en, multiple. 
df = df[~((df['url'] == '2897172.txt'))]  # two fr same, no en, multiple.
print("Number of rows after manually removing some files:", len(df))

# manually fix some language issues
df.loc[df['url'] == '1849447.txt', 'language'] = 'fr'
df.loc[df['url'] == '1529926.txt', 'language'] = 'fr'
df.loc[df['url'] == '1831945.txt', 'language'] = 'fr'
df.loc[df['url'] == '1831957.txt', 'language'] = 'fr'
df.loc[df['url'] == '1698426.txt', 'language'] = 'fr'
df.loc[df['url'] == '1526185.txt', 'language'] = 'fr'
df.loc[df['url'] == '1691141.txt', 'language'] = 'fr'
df.loc[df['url'] == '1691145.txt', 'language'] = 'fr'
df.loc[df['url'] == '1849475.txt', 'language'] = 'fr'

# fix dates format
df['document_date'] = pd.to_datetime(df['document_date'], errors='coerce', utc=True)

# Sort by document_date descending, so newest dates are first; NaT is last
df = df.sort_values(by=['document_date', 'scraped_timestamp'], ascending=[False, False])

# print number of rows with no document_date
print("Number of rows with no document_date:", df['document_date'].isna().sum())


# export to parquet the rows where there is no document_date
df_no_date = df[df['document_date'].isna()]
df_no_date.to_parquet("DATA/df_no_date.parquet", index=False)

# Remove rows where document_date is before 2013
print("Before removing before 2013", len(df))
# print the rows where df[df['document_date'] >= pd.Timestamp('2013-01-01', tz='UTC')].reset_index(drop=True)
print(df[df['document_date'] < pd.Timestamp('2013-01-01', tz='UTC')]['url'].tolist())

df = df[df['document_date'] >= pd.Timestamp('2013-01-01', tz='UTC')].reset_index(drop=True)

print ("After removing before 2013", len(df)) 

# reset index
df = df.reset_index(drop=True)

df




Starting df 29440
Number of rows after manually removing some files: 29433
Number of rows with no document_date: 0
Before removing before 2013 29433
[]
After removing before 2013 29433


,citation,citation2,dataset,name,language,document_date,url,scraped_timestamp,unofficial_text
0,MC3-00542,,RAD,,fr,2024-06-14 00:00:00+00:00,2. MC3-00542tf.txt,2024-09-10 01:13:43.447176+00:00,\nDossier de la SAR / RAD File: MC3-00542\n\nH...
1,MC3-00542,,RAD,,en,2024-06-14 00:00:00+00:00,1. MC3-00542 (Ouellet - Mexico - 111(1)(a) - D...,2024-09-10 01:13:43.141365+00:00,\nRAD File / Dossier de la SAR : MC3-00542\n\n...
2,VC3-14129,,RAD,,fr,2024-03-04 00:00:00+00:00,VC3-14129tf.txt,2024-07-12 18:44:44.255575+00:00,\nDossier de la SAR / RAD File: VC3-14129\n\nH...
3,VC3-14129,,RAD,,en,2024-03-04 00:00:00+00:00,VC3-14129 a.txt,2024-07-12 18:44:43.974644+00:00,\nRAD File / Dossier de la SAR : VC3-14129\n\n...
4,VC2-05699,,RAD,,fr,2024-02-21 00:00:00+00:00,VC2-05699tf.txt,2024-07-12 18:44:42.078996+00:00,\nDossier de la SAR / RAD File: VC2-05699\n\nH...
...,...,...,...,...,...,...,...,...,...
29428,VB3-00460,,RAD,,fr,2013-02-27 00:00:00+00:00,1519098.txt,2023-11-13 01:12:53.391303+00:00,\n\n\n\nN° de dossier de la SAR / RAD File No....
29429,VB3-00316,,RAD,,fr,2013-02-26 00:00:00+00:00,1526727.txt,2023-11-13 01:46:02.264031+00:00,\n\n\n\nN° de dossier de la SAR / RAD File No....
29430,VB3-00316,,RAD,,en,2013-02-26 00:00:00+00:00,1526685.txt,2023-11-13 01:16:17.679784+00:00,\n\n\n\nRAD File No. / N° de dossier de la SAR...
29431,VB3-00389,,RAD,,en,2013-02-19 00:00:00+00:00,1526712.txt,2023-11-13 01:46:01.151671+00:00,\n\n\n\nRAD File No. / N° de dossier de la SAR...


In [13]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # # Remove single newlines
    # text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

tqdm.pandas()
df['unofficial_text'] = df.unofficial_text.progress_apply(clean_text)

100%|██████████| 29433/29433 [00:40<00:00, 731.61it/s]


In [14]:
# get df_en where language is English
df_en = df[df['language'] == 'en'].reset_index(drop=True)
# get df_fr where language is French
df_fr = df[df['language'] == 'fr'].reset_index(drop=True)

# print how many df_en.citation is duplicated
print("Number of duplicated citations in df_en:", df_en.citation.duplicated().sum())
print("Number of duplicated citations in df_fr:", df_fr.citation.duplicated().sum())

# print how many rows in df_en are duplicated for BOTH citation and document_date
print("Number of duplicated rows in df_en for BOTH citation and document_date:", df_en.duplicated(subset=['citation', 'document_date']).sum())
print("Number of duplicated rows in df_fr for BOTH citation and document_date:", df_fr.duplicated(subset=['citation', 'document_date']).sum())

# export to parquet the rows where there are duplicated citations in df_en
df_en_duplicates = df_en[df_en.duplicated(subset=['citation'], keep=False)]
df_en_duplicates.to_parquet("DATA/df_en_duplicates.parquet", index=False)
# export to parquet the rows where there are duplicated citations in df_fr
df_fr_duplicates = df_fr[df_fr.duplicated(subset=['citation'], keep=False)]
df_fr_duplicates.to_parquet("DATA/df_fr_duplicates.parquet", index=False)   

# Blunt removal of duplicates in df_en and df_fr
df_en = df_en.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_en = df_en.drop_duplicates(subset=['unofficial_text'], keep='first').reset_index(drop=True)
df_en = df_en.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_en = df_en.drop_duplicates(subset=['citation'], keep='first').reset_index(drop=True)

df_fr = df_fr.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_fr = df_fr.drop_duplicates(subset=['unofficial_text'], keep='first').reset_index(drop=True)
df_fr = df_fr.sort_values(by=['citation', 'document_date', 'scraped_timestamp'], ascending=[True, False, False])
df_fr = df_fr.drop_duplicates(subset=['citation'], keep='first').reset_index(drop=True)

# Print the number of rows after removing duplicates
print("Number of rows in df_en after removing all duplicate citations:", len(df_en))
print("Number of rows in df_fr after removing duplicate citations:", len(df_fr))


# number of rows in df_en that does not have an identical citation in df_fr
print(
    "Num rows in df_en with no citation match in df_fr",
    df_en[~df_en['citation'].isin(df_fr['citation'])].reset_index(drop=True).shape[0]
)
# number of rows in df_fr that does not have an identical citation in df_en
print(
    "Num rows in df_fr with no citation match in df_en",
    df_fr[~df_fr['citation'].isin(df_en['citation'])].reset_index(drop=True).shape[0]
)



# for df_en add "_en" to end of all cols
df_en.columns = [col + "_en" for col in df_en.columns]
# for df_fr add "_fr" to end of all cols
df_fr.columns = [col + "_fr" for col in df_fr.columns]

# merge df_en and df_fr on citation_en and citation_fr, keeping only matches
df = pd.merge(df_en, df_fr, left_on='citation_en', right_on='citation_fr', how='inner')

# rename dataset_en to dataset, and drop unused cols
df = df.rename(columns={'dataset_en': 'dataset'})
df = df.drop(columns=['dataset_fr', 'language_en', 'language_fr'])


# print len of final df
print("Number of rows in final df:", len(df))
print("*********** AT SOME POINT RECONSIDER THIS ***********")
df.head(5)

Number of duplicated citations in df_en: 808
Number of duplicated citations in df_fr: 808
Number of duplicated rows in df_en for BOTH citation and document_date: 233
Number of duplicated rows in df_fr for BOTH citation and document_date: 235
Number of rows in df_en after removing all duplicate citations: 13906
Number of rows in df_fr after removing duplicate citations: 13910
Num rows in df_en with no citation match in df_fr 28
Num rows in df_fr with no citation match in df_en 32
Number of rows in final df: 13878
*********** AT SOME POINT RECONSIDER THIS ***********


,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,unofficial_text_fr
0,MB3-00631,,RAD,,2013-03-04 00:00:00+00:00,1506095.txt,2023-11-13 01:11:22.214474+00:00,\nImmigration and\nRefugee Board of Canada\nRe...,MB3-00631,,,2013-03-04 00:00:00+00:00,1506092.txt,2023-11-13 01:11:22.014576+00:00,\nCommission de l'immigration et du statut de ...
1,MB3-00772,,RAD,,2013-03-13 00:00:00+00:00,1506098.txt,2023-11-13 01:11:22.347155+00:00,\nImmigration and\nRefugee Board of Canada\nRe...,MB3-00772,,,2013-03-13 00:00:00+00:00,1506097.txt,2023-11-13 01:11:22.281050+00:00,\nCommission de l'immigration et du statut de ...
2,MB3-00899,,RAD,,2013-03-21 00:00:00+00:00,1415668.txt,2023-11-13 01:10:09.061743+00:00,\nImmigration and\nRefugee Board of Canada\nRe...,MB3-00899,,,2013-03-21 00:00:00+00:00,1415666.txt,2023-11-13 01:10:08.900619+00:00,\nCommission de l'immigration et du statut de ...
3,MB3-00900,,RAD,,2013-03-21 00:00:00+00:00,1507941.txt,2023-11-13 01:11:58.181480+00:00,\nRAD File No. / N° de dossier de la SAR : MB3...,MB3-00900,,,2013-03-21 00:00:00+00:00,1507922.txt,2023-11-13 01:11:57.481938+00:00,\nCommission de l'immigration et du statut de ...
4,MB3-01185,,RAD,,2013-07-09 00:00:00+00:00,1414390.txt,2023-11-13 01:10:06.703031+00:00,\nRAD File No. / N° de dossier de la SAR : MB3...,MB3-01185,,,2013-07-09 00:00:00+00:00,1414397.txt,2023-11-13 01:10:06.914564+00:00,\nN° de dossier de la SAR/RAD File No.: MB3-01...


In [15]:
# how many rows in df have document_date_en and document_date_fr that are not the same
print("Number of rows in df with different document_date_en and document_date_fr:", 
      df[df['document_date_en'] != df['document_date_fr']].shape[0])

Number of rows in df with different document_date_en and document_date_fr: 0


In [16]:
df[df['document_date_en'] != df['document_date_fr']]

,citation_en,citation2_en,dataset,name_en,document_date_en,url_en,scraped_timestamp_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,unofficial_text_fr


In [ ]:
# NOTE: CHECKED THIS ONE, MISSING: 
# MB3-02526	 
# MB7-05420 
# MB7-07448
# MB7-09115
# MB7-15795	### THERE'S A BUNCH FROM 2019 THAT ARE MISSING FROM MY FILES
# MB7-20800	
# MB7-21542	
# MB7-22590	
# MB7-24208	
# MB8-01099
# MB8-03922	
# TB4-00097
# TB4-00557
# TB4-03241	
# TB6-11497	
# TB8‑02686
# TB8-05513	
# TB8-08621
# TB8-13027
# TB8-18196
# TB8-19024
# TB8-32262
# TB9‑09752
# TB9-16802
# VB5-02697
# VB8-04489	
# VB9-02244	

# show rows where df_en.url is not in df.url_en
df_en[~df_en['url_en'].isin(df['url_en'])].reset_index(drop=True)



In [ ]:
# show citation_fr where df_fr.url_fr is not in df.url_fr
df_fr[df_fr['url_fr'].isin(df['url_fr']) == False].reset_index(drop=True)

In [ ]:
# show rows where df_fr.url is not in df.url_fr
fr_list=df_fr[~df_fr['url_fr'].isin(df['url_fr'])].reset_index(drop=True)['citation_fr'].to_list()

In [ ]:
import difflib
fr_list=df_fr[~df_fr['url_fr'].isin(df['url_fr'])].reset_index(drop=True)['citation_fr'].to_list()
en_list=df_en[~df_en['url_en'].isin(df['url_en'])].reset_index(drop=True)['citation_en'].to_list()
matches = []
cutoff = 0.8
for en_code in en_list:
    close = difflib.get_close_matches(en_code, fr_list, n=1, cutoff=cutoff)
    if close:
        ratio = difflib.SequenceMatcher(None, en_code, close[0]).ratio()
        matches.append((en_code, close[0], round(ratio, 2)))
df_matches = pd.DataFrame(matches, columns=['citation_en', 'citation_fr_candidate', 'similarity'])
df_matches = df_matches.sort_values('similarity', ascending=False)

# Print or export
print(df_matches)

In [17]:
# are there any duplicate citations_en in df?
print("Number of duplicated citations_en in df:", df.citation_en.duplicated().sum())
# are there any duplicate citations_fr in df?
print("Number of duplicated citations_fr in df:", df.citation_fr.duplicated().sum())

# are there any blank or null or nan citations_en in df?
print("Number of blank or null or nan citations_en in df:", df['citation_en'].isnull().sum())
# are there any blank or null or nan citations_fr in df?
print("Number of blank or null or nan citations_fr in df:", df['citation_fr'].isnull().sum())

Number of duplicated citations_en in df: 0
Number of duplicated citations_fr in df: 0
Number of blank or null or nan citations_en in df: 0
Number of blank or null or nan citations_fr in df: 0


### Exports

In [ ]:

# export cleaned df to parquet
out_path_parsed = pathlib.Path('DATA/rpd_cases.parquet')
df.to_parquet(out_path_parsed, index=False)

In [18]:
# export to Mongo # NEED TAILSCALE ON
from pymongo import MongoClient
from datetime import timezone
import numpy as np

MONGO_URL = 'mongodb://rll-tr7970x:27017/'
MONGO_DB = 'case-scraping'
MONGO_COLLECTION = 'rad-scrapes'
client = MongoClient(MONGO_URL, tz_aware=True, tzinfo=timezone.utc)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

# drop existing collection if it exists
if collection.count_documents({}) > 0:
    collection.drop()

def strip_nulls(recs):
    return [{k: v for k, v in rec.items() if v is not None} for rec in recs]

def sanitize_records(recs):
    """Return a NEW list of dicts that PyMongo can store."""
    return [
        {k: _clean_scalar(v) for k, v in rec.items()}
        for rec in recs
    ]

def _clean_scalar(v):
    # 1) All “missing” markers: <NA>, NaN, NaT  →  None
    if pd.isna(v):
        return None
    
    # 2) pandas Timestamp  →  native datetime with UTC tzinfo
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime().astimezone(timezone.utc)
    
    # 3) NumPy scalar (np.int64, np.float64, …)  →  Python int/float
    if isinstance(v, np.generic):
        return v.item()
    
    return v

records = df.to_dict(orient="records")
clean_docs = strip_nulls(sanitize_records(records))
result = collection.insert_many(clean_docs)


TOOLS

In [ ]:
# # To compare duplicates
# import difflib

# # Get the two texts
# texts = df_fr[df_fr.duplicated(subset=['citation', 'document_date'], keep=False)].unofficial_text.iloc[[0,1]]

# # Compute the diff
# diff = difflib.unified_diff(
#     texts.iloc[0].splitlines(),
#     texts.iloc[1].splitlines(),
#     fromfile='text0',
#     tofile='text1',
#     lineterm=''
# )

# # Print the diff
# for line in diff:
#     print(line)

In [ ]:
# # Create a Dask Bag from file paths
# file_bag = db.from_sequence(file_paths)

# def get_text(file_path):
#     """Read the file and return its content."""
#     with open(file_path, 'r', errors='replace', encoding='utf-8') as file:
#         file_name = os.path.basename(file_path)
#         return {"file": file_name, "text": file.read()}

# # Use Dask to process files in parallel
# with ProgressBar():
#     results = file_bag.map(get_text).filter(lambda x: x is not None).compute()

# # Convert results to a Pandas DataFrame
# df_all = pd.DataFrame(results)

In [ ]:
# # check if "SAR File" is in df_all.text
# df_all['has_sar_file'] = df_all['text'].str.contains("SAR File", na=False)

# # len of df_all where has_sar_file is True
# print("Number of rows in df_all with has_sar_file True:", df_all[df_all['has_sar_file']].shape[0])

# df_all[df_all['has_sar_file']].head(20)

# NOTES:

Note that the try/except print doesn't necessarily work if running in jupyter due to limitations 
in DASK. So if you need to know whether there are errors, uncomment the raise error thing.


TC1-10832tf.txt throws an error for some reason not sure why.